# 인바운드 인증

AgentCore Identity를 사용하면 AgentCore Runtime에서 에이전트 또는 도구를 호출하는 사용자 및 애플리케이션의 인바운드 액세스(인바운드 인증)를 검증하거나 AgentCore Gateway 대상에 대한 액세스를 검증할 수 있습니다. 또한 에이전트에서 외부 서비스 또는 Gateway 대상으로의 안전한 아웃바운드 액세스(아웃바운드 인증)를 제공합니다. 기존 ID 공급자(예: Amazon Cognito)와 통합되는 동시에 독립적으로 또는 사용자를 대신하여(OAuth를 통해) 에이전트에 대한 권한 경계를 적용합니다.

인바운드 인증은 AgentCore Runtime, AgentCore Gateway 또는 기타 환경에서 호스팅되는 에이전트 또는 도구를 호출하려는 호출자의 유효성을 검사합니다. 인바운드 인증은 IAM(SigV4 자격 증명) 또는 OAuth 권한 부여와 함께 작동합니다.

기본적으로 Amazon Bedrock AgentCore는 IAM 자격 증명을 사용하므로 에이전트에 대한 사용자 요청은 사용자의 IAM 자격 증명으로 인증됩니다. OAuth를 사용하는 경우 AgentCore 런타임 리소스 또는 AgentCore 게이트웨이 엔드포인트를 구성할 때 다음을 지정해야 합니다.

- OAuth 검색 서버 URL — OpenID Connect 검색 URL의 ^.+/\.well-known/openid-configuration$ 패턴과 일치해야 하는 문자열

- 허용된 대상 — JWT 토큰에 허용되는 대상 목록

- 허용된 클라이언트 — 허용되는 클라이언트 식별자 목록

AgentCore CLI를 사용하는 경우 **configure** 명령을 사용하여 AgentCore 런타임의 권한 부여 유형(및 OAuth 검색 서버)을 지정할 수 있습니다. CreateAgentRuntime 작업과 Amazon Bedrock AgentCore 콘솔을 사용할 수도 있습니다. 게이트웨이를 생성하는 경우 CreateGateway 작업 또는 콘솔을 사용합니다.

사용자가 에이전트를 사용하려면 클라이언트 애플리케이션에서 OAuth 권한 부여자를 통해 사용자를 인증해야 합니다. 클라이언트는 베어러 토큰을 수신한 후 호출 요청에서 에이전트에 전달합니다. 토큰을 수신하면 에이전트는 액세스를 허용하기 전에 권한 부여 서버에서 토큰의 유효성을 검사합니다.

## 개요

이 튜토리얼에서는 01-AgentCore-runtime에 배포한 에이전트를 수정하고 Cognito를 ID 공급자로 사용하여 인바운드 인증을 구성합니다. 사용자 한 명과 앱 클라이언트 한 개로 구성된 Cognito 사용자 풀을 설정합니다. Cognito 사용자 풀을 사용하는 인바운드 인증 기능을 갖춘 Amazon Bedrock AgentCore Runtime을 사용하여 기존 에이전트를 호스팅하는 방법을 알아봅니다.

### 튜토리얼 아키텍처

<div style="text-align:center">
<img src="images/inbound_auth_cognito.png" width="90%"/>
</div>

### 튜토리얼 세부 정보

| 정보 | 세부 정보 |
|:--------------------|:-------------------------------------------------------------------------------------------|
| 튜토리얼 유형 | 대화형 |
| 에이전트 유형 | 단일 |
| 에이전트 프레임워크 | 스트랜드 에이전트 |
| LLM 모델 | Anthropic Claude Sonnet 3.7 |
| 튜토리얼 구성 요소 | AgentCore 런타임에서 에이전트 호스팅. Strands 에이전트 및 Amazon Bedrock 모델 사용 |
| 튜토리얼 수직 | 교차 수직 |
| 예시 복잡성 | 쉬움 |
| 인바운드 인증 | Cognito |
| 사용 SDK | Amazon BedrockAgentCore Python SDK 및 boto3 |

### 튜토리얼 주요 기능

* Amazon Cognito를 사용하여 인바운드 인증을 통해 Amazon Bedrock AgentCore 런타임에서 에이전트 호스팅
* Amazon Bedrock 모델 사용
* Strands 에이전트 사용

## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker running

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Amazon Cognito 인증 설정

앱 클라이언트와 테스트 사용자 한 명으로 Cognito 사용자 풀을 프로비저닝해 보겠습니다. Amazon Cognito를 사용하여 배포된 MCP 서버에 액세스하기 위한 JWT 토큰을 제공합니다. 이를 위해 `utils` 스크립트의 `setup_cognito_user_pool` 지원 함수를 사용합니다.

참고: Cognito 액세스 토큰은 2시간 동안만 유효합니다. 액세스 토큰이 만료되면 `reauthenticate_user` 메서드를 사용하여 다른 액세스 토큰을 발급받을 수 있습니다.

In [ ]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import setup_cognito_user_pool, reauthenticate_user

In [ ]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")

## Preparing your agent for deployment on AgentCore Runtime

### Strands Agents with Amazon Bedrock model
Let's start with our Strands Agent we created in the 01-AgentCore-runtime tutorial and configure it with Inbound Auth that uses Amazon Cognito as the Identity Provider.

In [ ]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel

app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

## AgentCore 런타임에 에이전트 배포

`CreateAgentRuntime` 작업은 포괄적인 구성 옵션을 지원하여 컨테이너 이미지, 환경 변수 및 암호화 설정을 지정할 수 있습니다. 또한 프로토콜 설정(HTTP, MCP) 및 권한 부여 메커니즘을 구성하여 클라이언트가 에이전트와 통신하는 방식을 제어할 수 있습니다.

**참고:** 작업 모범 사례는 코드를 컨테이너로 패키징하고 CI/CD 파이프라인 및 IaC를 사용하여 ECR에 푸시하는 것입니다.

이 튜토리얼에서는 Amazon Bedrock AgentCode Python SDK를 사용하여 아티팩트를 쉽게 패키징하고 AgentCore 런타임에 배포할 수 있습니다.

### AgentCore 런타임 배포 구성

다음으로, 시작 툴킷을 사용하여 진입점, 방금 생성한 실행 역할, 그리고 요구 사항 파일을 사용하여 AgentCore 런타임 배포를 구성합니다. 또한 시작 툴킷이 실행 시 Amazon ECR 저장소를 자동으로 생성하도록 구성합니다.

구성 단계에서 애플리케이션 코드를 기반으로 Docker 파일이 생성됩니다.

**중요** - 이전 단계에서 설정한 Cognito Discovery URL과 Cognito 앱 클라이언트 ID를 업데이트하세요.

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

discovery_url = cognito_config.get("discovery_url")

client_id = cognito_config.get("client_id")

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_agent_inbound_identity",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,
            "allowedClients": [client_id]
        }
    }
)
response


## Review the AgentCore configuration

In [ ]:
!cat .bedrock_agentcore.yaml

### AgentCore 런타임에 에이전트 실행

이제 Docker 파일이 생성되었으니 AgentCore 런타임에 에이전트를 실행해 보겠습니다. 이렇게 하면 Amazon ECR 저장소와 AgentCore 런타임이 생성됩니다.

<div style="text-align:left">
<img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### 권한 없이 AgentCore 런타임 호출

마지막으로, 페이로드를 사용하여 AgentCore 런타임을 호출할 수 있습니다. 다음 셀을 실행해 보면 **"AccessDeniedException: InvokeAgentRuntime 작업을 호출하는 동안 오류가 발생했습니다(AccessDeniedException). 에이전트가 다른 권한 토큰 유형으로 구성되었습니다."**라는 오류가 표시됩니다.

<div style="text-align:left">
<img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now?"})
invoke_response

### 권한 부여를 사용하여 AgentCore 런타임 호출

올바른 권한 부여 토큰 유형을 사용하여 에이전트를 호출해 보겠습니다. 이 경우에는 Cognito 액세스 토큰을 사용합니다. "**Cognito 사용자 풀 프로비저닝**" 셀에서 액세스 토큰을 복사합니다.

In [ ]:
bearer_token = reauthenticate_user(cognito_config.get("client_id"))
invoke_response = agentcore_runtime.invoke(
    {"prompt": "How is the weather now?"}, 
    bearer_token=bearer_token
)
invoke_response

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
from boto3.session import Session
import boto3
boto_session = Session()

agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Congratulations!